In [2]:
from pprint import pprint
from collections import defaultdict
from timeit import timeit
from tqdm import tqdm

In [ ]:
F.<a> = GF(7)
R.<x> = PolynomialRing(F)
p = F.characteristic()
r = F.degree()
AGL = [ a * x + b for a in F if a != 0 for b in F ]

In [1]:
def numEquivClass(p, r, n): 
    """
    Given (p, r, n) as input, returns the number of AGL-equivalence classes of 
    polynomials with degree n in F_q where q = p^r. 

    Assumes that p is prime. 
    """
    q = p^r

    if n % p == 0:
        c = 1 - q^(n-2) + q^(n/p - 1)
    elif n == 1:
        c = 1
    else:
        c = 1 - q^(n-2)
    
    return 1/(q-1) * sum([euler_phi(d)*(q^(ceil(n/d)-1)-1) for d in divisors(q-1) if d < n]) + c

def numEquivClassF(F, deg): 
    """
    Given (F_q, deg) as input, returns the number of AGL-equivalence classes of 
    polynomials with degree deg in F_q. 
    """
    return numEquivClass(F.characteristic(), F.degree(), deg)

In [ ]:

len(AGL)

20

In [ ]:
def equivalent(p, q):
    for alpha in AGL:
        for beta in AGL:
            if q == alpha(p(beta)):
                return True
    return 

def normalize(p):
    lead = p.list()[-1]
    last = p.list()[0]
    return (p - last) / lead

def candidate_equivalent(p, q):
    for beta in AGL:
        if q == normalize(p(beta)):
            return True
    return False

In [7]:
def polys(deg, coeffs=()):
    if deg == 0:
        yield sum([ci * x^i for i, ci in enumerate(coeffs + (1,))])
        return
    for c in F:
        yield from polys(deg - 1, coeffs + (c,))

def candidates(deg, coeffs=()):
    if deg == 1:
        yield sum([ci * x^i for i, ci in enumerate((0,) + coeffs + (1,))])
        return
    for c in F:
        yield from candidates(deg - 1, coeffs + (c,))

In [ ]:
reps = set()
for p in candidates(5):
    if not any(candidate_equivalent(p, rep) for rep in reps):
        reps.add(p)
# pprint(reps)
print(len(reps))

18


In [7]:
numEquivClassF(GF(3), 6)

48

In [ ]:
def homogeneous(F, pdeg, max_deg):
    p = F.characteristic()
    pdeg_monomials = []
    max_pow = floor(log(max_deg, p))
    for partition in Partitions(pdeg, max_part=p-1, max_length=max_pow):
        for subset in Subsets(range(1, max_pow+1), len(partition)):
            exponent = sum(partition[i] * p^power for i, power in enumerate(subset))
            pdeg_monomials.append(x^exponent)
    return list(map(sum, cartesian_product([[a * monomial for a in F] for monomial in pdeg_monomials])))

In [ ]:
list(homogeneous(F, pdeg=1, max_deg=9))

[0, x^9, 2*x^9, x^3, x^9 + x^3, 2*x^9 + x^3, 2*x^3, x^9 + 2*x^3, 2*x^9 + 2*x^3]

In [4]:
def weight(n, p):
    w = 0
    while n > 0:
        w += n % p
        n //= p
    return w

def part(f, w, p):
    return sum(c * x^i for i, c in enumerate(f) if weight(i, p) == w)

def part_ge(f, w, p):
    return sum(c * x^i for i, c in enumerate(f) if weight(i, p) >= w)

def pdeg(f, p):
    return max([i for (i, c) in enumerate(f) if c != 0], key=lambda x: (weight(x, p), x))

In [ ]:
f = x^9 + x^6 + 2*x^5 + 2*x^2 + 1
print(part(f, 1, p))
print(pdeg(f, p))
print(list(map(lambda x: weight(x, p), f.exponents())))

2*x^5
9
[0, 2, 1, 2, 5]


In [ ]:
g = 2*x^29 + x^19 + x^17 + x^15 + x^14 + 2*x^8 + x^5
print(g(x + 2) - g)
print(list(map(lambda x: weight(x, p), g.exponents())))
print(list(map(lambda x: weight(x, p), (g(x + 2) - g).exponents())))

x^28 + 3*x^27 + 4*x^26 + 2*x^25 + 3*x^18 + 4*x^17 + x^16 + x^14 + x^13 + 4*x^12 + 3*x^11 + 2*x^10 + x^9 + 3*x^8 + 3*x^7 + 4*x^6 + 2*x^5 + x^4 + 2*x^3 + x^2 + 2*x
[3, 4, 4, 3, 5, 3, 3]
[1, 2, 1, 2, 3, 2, 3, 4, 1, 2, 3, 2, 3, 4, 4, 5, 2, 5, 6, 1, 2]


In [6]:
def canonical_form(f, p):
    d = pdeg(f, p)
    f /= f[d]
    f -= f[0]
    K = weight(d, p)

    form = 0
    f_partial = 0
    f_parts = [part(f, i, p) for i in range(K + 1)]

    candidates = AGL
    # print(len(candidates))
    for k in range(K, 0, -1):
        f_partial += f_parts[k]
        g_options = defaultdict(list)
        for poly in candidates:
            g_options[part(poly[1]^(-d) * f_partial(poly), k, p)].append(poly)
        maximal = min(g_options)
        form += maximal
        candidates = g_options[maximal]
        # print(form)
        # print(len(candidates))
    return form

def canonical_form_hyp(f, p):
    d = pdeg(f, p)
    f /= f[d]
    f -= f[0]
    K = weight(d, p)

    g = part(f, K, p)
    a = min([a for a in F if a != 0], key=lambda a: a^(-d) * g(a * x))
    form = a^(-d) * g(a * x)
    poly = a * x
    for k in range(K - 1, 0, -1):
        # print("form:", form)
        # print("   f:", part_ge(poly[1]^(-d) * f(poly), k + 1, p))
        g = part(poly[1]^(-d) * f(poly), k, p)
        alphas = [a for a in F if a != 0 and form == a^(-d) * form(a * x)]
        ys = [y for y in F if part_ge(form(x + y) - form, k + 1, p) == 0]
        pairs = [(a, y) for a in alphas for y in ys]
        a, y = min(pairs, key=lambda pr: part(form(x + pr[1]) - form, k, p) + pr[0]^(-d) * g(pr[0] * x))
        form += part(form(x + y) - form, k, p) + a^(-d) * g(a * x)
        poly = poly(a * x + y * a)
    return form

In [ ]:
print(canonical_form(f, p))
print(canonical_form(f(x + 1), p))
print(canonical_form(f(x + (a^2 + 1)), p))
print(canonical_form((a + 3) * f(x + (a^2 + 1)) - 4, p))

x^9 + x^6 + 2*x^5 + 2*x^2
x^9 + x^6 + 2*x^5 + 2*x^2
x^9 + x^6 + 2*x^5 + 2*x^2
x^9 + x^6 + 2*x^5 + 2*x^2


In [ ]:
print(canonical_form_hyp(f, p))
print(canonical_form_hyp(f(x + 1), p))
print(canonical_form_hyp(f(x + (a^2 + 1)), p))
print(canonical_form_hyp((a + 3) * f(x + (a^2 + 1)) - 4, p))

x^9 + x^6 + 2*x^5 + 2*x^2
x^9 + x^6 + 2*x^5 + 2*x^2
x^9 + x^6 + 2*x^5 + 2*x^2
x^9 + x^6 + 2*x^5 + 2*x^2


In [ ]:
def random_polynomial(max_deg, F):
    return sum(F.random_element() * x^i for i in range(randint(2, max_deg)))

In [ ]:
results = []
for _ in range(10):
    f = random_polynomial(30, F)
    print("  f:", f)
    c1 = canonical_form(f, p)
    print("  1:", c1)
    c2 = canonical_form_hyp(f, p)
    print("  2:", c2)
    print("---:", c1 == c2)
    results.append(c1 == c2)
print("ALL TESTS PASSED" if all(results) else f"!!! {len(results) - sum(results)} FAILURES !!!")

  f: (2*a^2 + 4*a + 2)*x^24 + (a^2 + 1)*x^23 + (3*a^2 + 2*a + 1)*x^22 + (2*a^2 + 2*a)*x^21 + (4*a^2 + 3*a + 4)*x^20 + a*x^19 + (3*a^2 + 3*a + 3)*x^18 + (a^2 + 3)*x^17 + (4*a^2 + a + 1)*x^16 + (3*a^2 + 4)*x^15 + 2*a*x^14 + 4*a^2*x^13 + (2*a^2 + 3*a + 2)*x^12 + (a^2 + 4*a + 3)*x^11 + 4*x^10 + (3*a^2 + 4*a)*x^9 + (3*a^2 + 4*a + 1)*x^8 + (3*a^2 + 4*a + 2)*x^7 + (2*a^2 + 2)*x^6 + (2*a^2 + 4*a + 1)*x^5 + (3*a^2 + 3*a + 4)*x^4 + (3*a^2 + 2*a + 2)*x^3 + (4*a^2 + 3*a)*x^2 + (a^2 + 3*a)*x + 2*a + 2
  1: x^24 + (3*a^2 + 2*a + 4)*x^22 + (a^2 + 4*a + 4)*x^21 + a*x^20 + x^19 + 3*a*x^18 + (4*a + 4)*x^17 + (4*a^2 + 4*a)*x^16 + (3*a + 1)*x^15 + (2*a^2 + a + 1)*x^14 + 2*a^2*x^13 + a*x^12 + (3*a^2 + 4*a + 1)*x^11 + (3*a^2 + a)*x^10 + (3*a^2 + 4*a + 4)*x^9 + (2*a^2 + 2*a)*x^8 + (a^2 + a + 1)*x^7 + (3*a^2 + 3*a + 2)*x^6 + (3*a^2 + 3)*x^5 + (4*a + 4)*x^4 + (4*a + 4)*x^3 + 3*a^2*x^2 + (4*a + 2)*x
  2: x^24 + (3*a^2 + 2*a + 4)*x^22 + (a^2 + 4*a + 4)*x^21 + a*x^20 + x^19 + 3*a*x^18 + (4*a + 4)*x^17 + (4*a^2 + 

In [ ]:
test_poly = (2*a^2 + 3*a)*x^28 + (2*a + 4)*x^27 + (2*a^2 + 3*a + 4)*x^26 + (a^2 + 3*a + 4)*x^25 + (4*a^2 + 4*a + 1)*x^24 + (a^2 + 4*a + 4)*x^23 + (2*a^2 + a + 4)*x^22 + (3*a^2 + 3*a + 2)*x^21 + (a^2 + 4*a)*x^20 + (3*a^2 + 2*a)*x^19 + (2*a^2 + a)*x^18 + (3*a + 4)*x^17 + (3*a^2 + 3)*x^16 + (4*a + 2)*x^15 + (a^2 + 2*a + 3)*x^14 + (a^2 + a + 3)*x^13 + (2*a^2 + 3*a + 4)*x^12 + (a^2 + 4*a + 3)*x^11 + (3*a + 2)*x^10 + (a + 3)*x^9 + (4*a^2 + 3*a + 2)*x^8 + (a^2 + 3*a)*x^7 + (2*a^2 + 3*a + 1)*x^6 + (4*a^2 + a)*x^5 + (2*a^2 + 4*a + 4)*x^4 + (3*a^2 + 2*a + 2)*x^3 + (4*a^2 + 4*a + 2)*x^2 + (a^2 + 2*a + 4)*x + 4*a^2 + a + 1
print("#1:", timeit("canonical_form(test_poly, p)", number=5, repeat=5, globals=globals()))
print("#2:", timeit("canonical_form_hyp(test_poly, p)", number=5, repeat=5, globals=globals()))

#1: 5 loops, best of 5: 9.82 s per loop
#2: 5 loops, best of 5: 5.62 s per loop


In [ ]:
def polysOfDeg(n): 
    """returns all monic polynomials of deg exactly n that have constant term 0 as a set""" 
    out = []
    coeffs_set = cartesian_product([F for _ in range(n-1)])
    pbar = tqdm(coeffs_set)
    num_polys = 0 
    total_polys = F.order()
    for coeffs in pbar: 
        out.append(R([0] + list(coeffs) + [1]))
        num_polys += 1
        pbar.set_description(f"Finding polys: {num_polys} done out of {}")
    return out

def enumAllCanonicalForm(n):
    """enumerates all canonical forms of deg <= n by brute force search """
    forms = {}
    candidates = polysOfDeg(n)
    visited = set() 
    q = F.order()
    pbar = tqdm(candidates)
    polys_done = 0
    total_polys = len(candidates)
    total_equiv_classes = numEquivClass(p, r, n)
    for f in pbar:
        if f not in visited:
            form = canonical_form(f, p)
            d = pdeg(f, p)
            allothers = [a^(-d) * form(a * x + y) for a in F if a != 0 for y in F]
            constantcoeffs = {f - f[0] for f in allothers}
            forms[form] = constantcoeffs
            visited |= constantcoeffs
            if len(visited) == len(candidates): 
                break
        polys_done += 1
        pbar.set_description(f"Finding canonical forms: {polys_done} out of {total_polys} done, {len(forms)} out of {total_equiv_classes} found ")
    return forms

In [155]:
enumAllCanonicalForm(5)

57950 out of 390625 done, 682 out of 683 found :  15%|█▍        | 57950/390625 [3:23:24<19:27:41,  4.75it/s] 


KeyboardInterrupt: 